In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

In [2]:
def convert_dtype(df):
    df[df.columns[1:]] = df[df.columns[1:]].astype("float64")
    return df


def reset_index(df):
    df_index = pd.Index(range(df.shape[0]), dtype="int")
    df = df.set_index(df_index)
    return df

data_path = r"../data/raw/raw_data.xlsx"
na_values = ["b.d", "bdl", "b.d.l.", "BDL", "<LOD", "-", "b.d.l", "n.d.", "bdl "]
raw_df = pd.read_excel(data_path, na_values=na_values)
raw_df

,Deposit type,Co,Ni,Cu,Zn,As,Se,Ag,Sb,Te,Au,Pb,Bi
0,IOA,18100.00,NaN,6.65,100,1.1,NaN,NaN,NaN,0.33,0.19,NaN,NaN
1,IOA,14700.00,NaN,8.72,100,1.8,NaN,NaN,NaN,0.55,0.23,NaN,NaN
2,IOA,730.00,NaN,2930,9030,10.0,290,1.2,8.18,420,0.17,663,NaN
3,IOA,2900.00,8200.00,0.11,100,10.0,85,NaN,0.04,0.04,0.05,100,NaN
4,IOA,4100.00,400.00,1100,100,10.0,NaN,100,100,0.9,0.26,200,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3279,Sedimentary pyrite,40.58,408.21,274.32,18.48,450.0,215.5,102.51,120.25,0.52,<0.01,679.89,2.77
3280,Sedimentary pyrite,40.45,496.73,323.15,78.05,582.7,270.5,133.98,170.48,0.98,<0.01,1475.52,3.02
3281,Sedimentary pyrite,40.80,503.28,285.82,28.74,475.2,258.2,112.27,132.11,0.75,0.05,1066.09,3.13
3282,Sedimentary pyrite,49.32,454.38,338.29,60.71,686.8,274.8,31.9,116.1,1.45,<0.01,317.07,3.46


In [31]:
def knn_function(imputer, df):
    columns = df.columns
    not_object_cols = []
    object_and_nan_cols_index = []
    for i, col in enumerate(columns):
        if df[col].isnull().all() or df[col].dtype == object:
            object_and_nan_cols_index.append(i)
        else:
            not_object_cols.append(col)
    X = df[not_object_cols]
    X = pd.DataFrame(imputer.fit_transform(X))
    X.index = df.index
    for object_and_nan_col_index in object_and_nan_cols_index:
        object_and_nan_col = columns[object_and_nan_col_index]
        X.insert(object_and_nan_col_index, object_and_nan_col, df[object_and_nan_col])
    X.columns = columns
    return X


df_replace_less = raw_df.replace("<.*", np.nan, regex=True)
df_replace_less = convert_dtype(df_replace_less)

df_groups = df_replace_less.groupby("Deposit type")
df = pd.DataFrame()
n_neighbors = 5
imputer = KNNImputer(n_neighbors=n_neighbors)
for _, group_item in df_groups:
    X = knn_function(imputer, group_item)
    df = pd.concat([df, X], axis=0)

df = df.sort_index()
clean_data_path = "../data/clean/data_clean_with_knn.csv"
df.to_csv(clean_data_path, index=False)

In [69]:
X = [[np.nan, 2, np.nan, 6], [np.nan, 4, 3, 5], [np.nan, 6, 5, 2], [np.nan, 8, 10, 4]]
print(np.array(X))
imputer = KNNImputer(n_neighbors=3)
imputer.fit_transform(X)

[[nan  2. nan  6.]
 [nan  4.  3.  5.]
 [nan  6.  5.  2.]
 [nan  8. 10.  4.]]


array([[ 2.,  6.,  6.],
       [ 4.,  3.,  5.],
       [ 6.,  5.,  2.],
       [ 8., 10.,  4.]])

In [30]:
df_remove_less = raw_df.replace("<", "", regex=True)
print(df_remove_less)
print(df_remove_less.dtypes)
df_remove_less = convert_dtype(df_remove_less)
print(df_remove_less.dtypes)

    Deposit type            Co           Ni           Cu       Zn  \
2            IOA    730.000000          NaN  2930.000000  9030.00   
3           IOCG  14000.000000     2.400000     0.480000     1.40   
4  Skarn Cu-(Fe)      1.690000     1.260000     7.050000      NaN   
5    Orogenic Au     36.562541    47.125287    42.067685      NaN   
6            VMS    306.000000    72.300000   171.000000  1461.00   
7            VMS    667.200000   539.100000  3996.000000  5278.00   
8            VMS      2.900000    29.600000   644.600000     8.00   
9          SEDEX     54.380000  2262.310000  3590.350000    31.76   

            As          Se          Ag      Sb          Te        Au  \
2    10.000000  290.000000    1.200000    8.18  420.000000      0.17   
3    14.000000   68.470000         NaN     NaN    0.198516     0.013   
4    15.300000    0.770000    3.230000    0.18         NaN      0.05   
5   105.229273    1.485378   11.594954     NaN   11.320727  1.055383   
6  1511.000000    

In [25]:
# 根据具体缺失元素来筛选所需数据
# select_index = list(map(lambda x: not x, raw_df["Ni"].isnull() & raw_df["Se"].isnull()))
# raw_df = raw_df[select_index]

# 根据缺失元素数量来筛选所需数据
allow_null_number = 5
thresh = raw_df.shape[1] - allow_null_number
raw_df_dropna = raw_df.dropna(thresh=thresh)
raw_df_dropna